In [2]:
%%capture
!pip install openai
from openai import OpenAI
import json
from pydantic import BaseModel
from typing import List
from enum import Enum
import pandas as pd

In [3]:
APIKEY = "" # @param {type:"string"}

In [4]:
client = OpenAI(api_key=APIKEY)

In [5]:
class Genre(Enum):
    Fiction = "Fiction"
    Non_Fiction = "Non-Fiction"

In [6]:
class BookResponse(BaseModel):
    bookTitle: List[str]
    publishDate: List[int]
    genre: List[Genre]

In [7]:
AUTHOR = "James Baldwin" # @param {type:"string"}


In [8]:
completion = client.chat.completions.create(
  model="gpt-4-turbo-preview",
  messages=[
     {"role": "user", "content": f"""List of book titles authored by {AUTHOR}. Also include
     original publishing date as well as the genre (either fiction or non-fiction).
      """}
  ],
  functions=[{"name": "get_books_from_author",
              "description": """Get user answer in an array. The array must contain a list
              of books from the user input, as well as each corresponding original publishing
              date, and genre.""",
              "parameters": BookResponse.schema()}],
  function_call = {"name": "get_books_from_author"}
)

json_response = json.loads(completion.choices[0].message.function_call.arguments)

df = pd.DataFrame(json_response)

In [9]:
display(df)

,bookTitle,publishDate,genre
0,Go Tell It on the Mountain,1953,Fiction
1,Notes of a Native Son,1955,Non-Fiction
2,Giovanni’s Room,1956,Fiction
3,Another Country,1962,Fiction
4,The Fire Next Time,1963,Non-Fiction
5,No Name in the Street,1972,Non-Fiction
6,If Beale Street Could Talk,1974,Fiction
7,Just Above My Head,1979,Fiction
8,The Evidence of Things Not Seen,1985,Non-Fiction
9,The Devil Finds Work,1976,Non-Fiction
